In [1]:
import os
import numpy as np
import gym
from gym import wrappers
from PIL import Image
# import pybullet_envs

ENV_NAME = 'BipedalWalker-v2'
# ENV_NAME = 'HalfCheetahBulletEnv-v0'

In [2]:
class Hp():
    # Hyperparameters
    def __init__(self,
                 nb_steps=30000,
                 episode_length=2000,
                 learning_rate=0.02,
                 num_deltas=16,
                 num_best_deltas=16,
                 noise=0.03,
                 seed=1,
                 env_name='BipedalWalker-v2',
                 record_every=100):

        self.nb_steps = nb_steps
        self.episode_length = episode_length
        self.learning_rate = learning_rate
        self.num_deltas = num_deltas
        self.num_best_deltas = num_best_deltas
        assert self.num_best_deltas <= self.num_deltas
        self.noise = noise
        self.seed = seed
        self.env_name = env_name
        self.record_every = record_every


class Normalizer():
    # Normalizes the inputs
    def __init__(self, nb_inputs):
        self.n = np.zeros(nb_inputs)
        self.mean = np.zeros(nb_inputs)
        self.mean_diff = np.zeros(nb_inputs)
        self.var = np.zeros(nb_inputs)

    def observe(self, x):
        self.n += 1.0
        last_mean = self.mean.copy()
        self.mean += (x - self.mean) / self.n
        self.mean_diff += (x - last_mean) * (x - self.mean)
        self.var = (self.mean_diff / self.n).clip(min = 1e-2)

    def normalize(self, inputs):
        obs_mean = self.mean
        obs_std = np.sqrt(self.var)
        return (inputs - obs_mean) / obs_std


class Policy():
    def __init__(self, input_size, output_size, hp):
        self.theta = np.zeros((output_size, input_size))
        self.hp = hp

    def evaluate(self, input, delta = None, direction = None):
        if direction is None:
            return self.theta.dot(input)
        elif direction == "+":
            return (self.theta + self.hp.noise * delta).dot(input)
        elif direction == "-":
            return (self.theta - self.hp.noise * delta).dot(input)

    def sample_deltas(self):
        return [np.random.randn(*self.theta.shape) for _ in range(self.hp.num_deltas)]

    def update(self, rollouts, sigma_rewards):
        # sigma_rewards is the standard deviation of the rewards
        step = np.zeros(self.theta.shape)
        for r_pos, r_neg, delta in rollouts:
            step += (r_pos - r_neg) * delta
        self.theta += self.hp.learning_rate / (self.hp.num_best_deltas * sigma_rewards) * step


class ArsTrainer():
    def __init__(self,
                 hp=None,
                 input_size=None,
                 output_size=None,
                 normalizer=None,
                 policy=None,
                 monitor_dir=None):

        self.hp = hp or Hp()
        np.random.seed(self.hp.seed)
        self.env = gym.make(self.hp.env_name)
        if monitor_dir is not None:
            should_record = lambda i: self.record_video
            self.env = wrappers.Monitor(self.env, monitor_dir, video_callable=should_record, force=True)
        self.hp.episode_length = self.env.spec.timestep_limit or self.hp.episode_length
        self.input_size = input_size or self.env.observation_space.shape[0]
        self.output_size = output_size or self.env.action_space.shape[0]
        self.normalizer = normalizer or Normalizer(self.input_size)
        self.policy = policy or Policy(self.input_size, self.output_size, self.hp)
        self.record_video = False
        self.rewards_list = []
        self.log_interval = 10
        self.optimal_policy = None

    # Explore the policy on one specific direction and over one episode
    def explore(self, direction=None, delta=None):
        state = self.env.reset()
        done = False
        num_plays = 0.0
        sum_rewards = 0.0
        while not done and num_plays < self.hp.episode_length:
            self.normalizer.observe(state)
            state = self.normalizer.normalize(state)
            action = self.policy.evaluate(state, delta, direction)
            state, reward, done, _ = self.env.step(action)
            reward = max(min(reward, 1), -1)
            sum_rewards += reward
            num_plays += 1
        return sum_rewards

    def train(self):
        for step in range(self.hp.nb_steps):
            # initialize the random noise deltas and the positive/negative rewards
            deltas = self.policy.sample_deltas()
            positive_rewards = [0] * self.hp.num_deltas
            negative_rewards = [0] * self.hp.num_deltas

            # play an episode each with positive deltas and negative deltas, collect rewards
            for k in range(self.hp.num_deltas):
                positive_rewards[k] = self.explore(direction="+", delta=deltas[k])
                negative_rewards[k] = self.explore(direction="-", delta=deltas[k])
                
            # Compute the standard deviation of all rewards
            sigma_rewards = np.array(positive_rewards + negative_rewards).std()

            # Sort the rollouts by the max(r_pos, r_neg) and select the deltas with best rewards
            scores = {k:max(r_pos, r_neg) for k,(r_pos,r_neg) in enumerate(zip(positive_rewards, negative_rewards))}
            order = sorted(scores.keys(), key = lambda x:scores[x], reverse = True)[:self.hp.num_best_deltas]
            rollouts = [(positive_rewards[k], negative_rewards[k], deltas[k]) for k in order]

            # Update the policy
            self.policy.update(rollouts, sigma_rewards)
                        
            # Only record video during evaluation, every n steps
            if step % self.hp.record_every == 0:
                self.record_video = True
                
            # Play an episode with the new weights and print the score
            reward_evaluation = self.explore()
            
            self.rewards_list.append(reward_evaluation)
            avg_reward = np.mean(self.rewards_list[-100:])
            
            if step % self.log_interval == 0:
                print('Step: {}  reward: {:4.2f} avg_reward: {:4.2f}  sigma: {:4.2f}  video: {}'.format(step, reward_evaluation, avg_reward, sigma_rewards, self.record_video))
                
            self.record_video = False
            
            # if avg reward > 300 then save and stop traning:
            if avg_reward >= self.env.spec.reward_threshold: 
                print("########## Solved! ###########")        
                self.optimal_policy = self.policy
                break
                
    def test(self, direction=None, delta=None):
        n_episodes = 3
        save_gif = True
        env = gym.make(self.hp.env_name)
        
        for ep in range(1, n_episodes+1):
            state = env.reset()
            done = False
            num_plays = 0.0
            ep_reward = 0.0
            
            for t in range(self.hp.episode_length):
                self.normalizer.observe(state)
                state = self.normalizer.normalize(state)
                action = self.optimal_policy.evaluate(state, delta, direction)
                state, reward, done, _ = env.step(action)
                reward = max(min(reward, 1), -1)
                ep_reward += reward
                num_plays += 1
                
                if save_gif:
                    dirname = './gif/ars/{}'.format(ep)
                    if not os.path.isdir(dirname):
                        os.mkdir(dirname)
                    img = env.render(mode = 'rgb_array')
                    img = Image.fromarray(img)
                    img.save('./gif/ars/{}/{}.jpg'.format(ep,t))
                if done:
                    break
                    
            print('Episode: {}\tReward: {}'.format(ep, int(ep_reward)))          
                   
            


def mkdir(base, name):
    path = os.path.join(base, name)
    if not os.path.exists(path):
        os.makedirs(path)
    return path


In [3]:
# Main code
videos_dir = mkdir('.', 'videos')
monitor_dir = mkdir(videos_dir, ENV_NAME)
hp = Hp(seed=1946, env_name=ENV_NAME)

trainer = ArsTrainer(hp=hp, monitor_dir=monitor_dir)
trainer.train()

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
Step: 0  reward: 4.75 avg_reward: 4.75  sigma: 10.61  video: True
Step: 10  reward: -0.05 avg_reward: -1.06  sigma: 8.58  video: False
Step: 20  reward: 3.03 avg_reward: -0.14  sigma: 7.01  video: False
Step: 30  reward: 3.41 avg_reward: 1.11  sigma: 4.19  video: False
Step: 40  reward: -9.47 avg_reward: 1.74  sigma: 2.28  video: False
Step: 50  reward: 2.77 avg_reward: 2.29  sigma: 7.37  video: False
Step: 60  reward: 3.38 avg_reward: 2.56  sigma: 8.86  video: False
Step: 70  reward: 4.15 avg_reward: 2.72  sigma: 1.35  video: False
Step: 80  reward: 4.37 avg_reward: 2.91  sigma: 7.26  video: False
Step: 90  reward: 4.78 avg_reward: 3.11  sigma: 0.91  video: False
Step: 100  reward: 4.83 avg_reward: 3.25  sigma: 0.58  video: True
Step: 110  reward: 5.62 avg_reward: 3.91  sigma: 0.92  video: 

In [5]:
trainer.test()

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
Episode: 1	Reward: 313
Episode: 2	Reward: 315
Episode: 3	Reward: 313
